In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ipydatagrid
!pip install pyhive[presto]
!pip install sqlalchemy-trino

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyhive: filename=PyHive-0.7.0-py3-none-any.whl size=53872 sha256=4d63deb9a16a5c103f24483ffc3f7dbe17bd908d844daf7317499fcefaa5ad55
  Stored in directory: /root/.cache/pip/wheels/d3/fc/31/6974270c69ccc5bf8f848e2e41b527d0e8f5b9b973696a29a9
Successfully built pyhive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import create_engine

In [ ]:
userid    = "ibmlhadmin"
password  = "password"
hostname  = "na4.services.cloud.techzone.ibm.com"
port      = "48700"
catalog   = "tpch"
schema    = "taxi"
certfile  = "/content/drive/My Drive/Uni AI/certs/lh-ssl-ts.crt"
connect_args={
        'protocol': 'https',
        'requests_kwargs': {'verify': f'{certfile}'}
        }


In [ ]:
# from sqlalchemy.dialects import create_engine

engine = create_engine(
   f"presto://{userid}:{password}@{hostname}:{port}/{catalog}/{schema}",
   connect_args=connect_args
   )

In [ ]:
mypresto = pd.read_sql_query('SELECT * from hive_data.taxi.taxirides limit 300',engine)

In [ ]:
mypresto

In [ ]:
# @title payment_type

from matplotlib import pyplot as plt
import seaborn as sns
mypresto.groupby('payment_type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
mypresto2 = pd.read_sql_query('SELECT * from hive_data.taxi.taxirides limit 10000',engine)

In [ ]:
taxirides=mypresto2

In [ ]:
taxirides

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

In [ ]:
# Example coordinates for point (a, b), replace with actual values
a, b = 41.978611, -87.904724  # New York City coordinates for example

# Calculate distances
taxirides['distance_to_ab'] = taxirides.apply(
    lambda row: haversine_distance(row['pickup_latitude'], row['pickup_longitude'], a, b),
    axis=1
)
# Summary statistics for distances
print(taxirides['distance_to_ab'].describe())

# Plotting the distribution of distances
import matplotlib.pyplot as plt

plt.hist(taxirides['distance_to_ab'], bins=30, edgecolor='black')
plt.title('Distribution of Distances from Point (a, b)')
plt.xlabel('Distance (km)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Example coordinates for point (a, b), replace with actual values
a, b = 41.978611, -87.904724

# Calculate distances
taxirides['distance_to_ab'] = taxirides.apply(
    lambda row: haversine_distance(row['dropoff_latitude'], row['dropoff_longitude'], a, b),
    axis=1
)
# Summary statistics for distances
print(taxirides['distance_to_ab'].describe())

# Plotting the distribution of distances
import matplotlib.pyplot as plt

plt.hist(taxirides['distance_to_ab'], bins=30, edgecolor='black')
plt.title('Distribution of Distances from Point (a, b)')
plt.xlabel('Distance (km)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
pip install folium

In [ ]:
import folium
import pandas as pd

from folium.plugins import MarkerCluster

# Example: Using a central coordinate, adjust as necessary
airport_latitude  = 41.978611  # New York City latitude
airport_longitude  = -87.904724 # New York City longitude

m = folium.Map(location=[central_latitude, central_longitude], zoom_start=12)

folium.Marker(
    [airport_latitude, airport_longitude],
    popup='JFK Airport',
    icon=folium.Icon(color='red', icon='plane', prefix='fa'),
    tooltip='Click for more info'
).add_to(m)

# Create a Marker Cluster
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame and add each pickup location as a marker
for index, row in taxirides_cleaned.iterrows():
    folium.Marker(
        location=[row['pickup_latitude'], row['pickup_longitude']],
        popup=f"Pickup Coordinates: ({row['pickup_latitude']}, {row['pickup_longitude']})",
    ).add_to(marker_cluster)


In [ ]:
m

In [ ]:
print(taxirides_cleaned.isna().sum())


In [ ]:
# Remove rows where either pickup_latitude or pickup_longitude is NaN
taxirides_cleaned = taxirides.dropna(subset=['pickup_latitude', 'pickup_longitude'])


In [ ]:
taxirides=taxirides_cleaned

In [ ]:
taxirides

In [ ]:
import pandas as pd
# Filter to keep only rows where distance_to_ab is 5 or less
taxirides_close = taxirides[taxirides['distance_to_ab'] <= 10]
taxirides_far= taxirides[taxirides['distance_to_ab'] >= 10]

In [ ]:
# Assuming taxirides is your DataFrame and 'fare' is the column name
average_fare_full = taxirides_far['trip_total'].mean()
# Assuming taxirides is your DataFrame and 'fare' is the column name
average_miles_full = taxirides_far['trip_miles'].mean()

print(f"The average fare is: {average_fare_full}")
print(f"The average fare is: {average_miles_full}")


In [ ]:
# Assuming taxirides is your DataFrame and 'fare' is the column name
average_fare_close = taxirides_close['trip_total'].mean()
# Assuming taxirides is your DataFrame and 'fare' is the column name
average_miles_close = taxirides_close['trip_miles'].mean()

print(f"The average fare is: {average_fare_close}")
print(f"The average fare is: {average_miles_close}")


# flights

In [ ]:
mypresto2 = pd.read_sql_query('SELECT * from hive_data.ontime.ontime limit 10000',engine)

In [ ]:
flight_info=mypresto2

In [ ]:
flight_info.head()

In [ ]:
import pandas as pd

# Assuming flight_info is your DataFrame
# Specify the columns to keep
columns_to_keep = [
    'ArrTime', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime',
    'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay',
    'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'
]

# Select only the specified columns
flight_info_filtered = flight_info[columns_to_keep]

# Now flight_info_filtered contains only the columns of interest


In [ ]:
flight_info['taxiout']